# S&P500 CFD Pepperstone Data Edge

## raw data
### import data

In [14]:
import pandas as pd 
import pandas_ta as ta
from edge_tools.ohlcv import normalize_ohlcv

# Load the data
# Ensure the path is correct and points to your CSV file
# Example: data_path = "./data/US500_Minute_20250821_0926.csv"
DATA_MINUTE_LONG = "./data/US500_Minute_20250821_0926.csv"
DATA_MINUTE_SHORT = "/Users/ducjeremyvu/trading/price_data/US500_Minute_20251104_0848.csv"

data_path = DATA_MINUTE_SHORT



# Read the CSV file
# Adjust the path as necessary
# If the file is large, consider using chunksize or dtypes to optimize memory usage
data = pd.read_csv(data_path)

# Convert 'Date' column to datetime and set as index, also normalizes header names
data = normalize_ohlcv(data, style="lowercase")



data


,time,open,high,low,close,volume
0,2025-10-28 08:46,6873.8,6873.8,6873.3,6873.8,24
1,2025-10-28 08:47,6874.0,6874.8,6874.0,6874.8,26
2,2025-10-28 08:48,6874.5,6874.8,6874.1,6874.5,12
3,2025-10-28 08:49,6874.3,6875.5,6874.3,6875.5,16
4,2025-10-28 08:50,6875.3,6876.8,6875.3,6876.5,26
...,...,...,...,...,...,...
6820,2025-11-04 08:44,6790.1,6792.2,6790.1,6791.4,96
6821,2025-11-04 08:45,6791.8,6792.4,6790.4,6790.7,90
6822,2025-11-04 08:46,6790.8,6791.8,6789.7,6790.2,116
6823,2025-11-04 08:47,6789.9,6790.7,6788.7,6789.4,90


## Problem Analysis 

I use data that's according to the timezone where I live in. That is Germany. In the summer it is UTC+2 and during winter it changes to UTC+1. Some places do not adopt such Day Time Saving (DTS) regime. Some other places adopt it a week later. It brings with a lot of confusions so it would make sense assigning the timezone parameter to such data objects to then being able to convert them to the needed timezones. With that I can target US Open regardless of my location, target Tokyo opens regardles off DTS. 

## Details 

 - data from Ctrader, CTrader with pepperstone always ran at UTC+2
 - so the idea is to assign the datetime with timezone parameter
 - the idea came when i wanted to see price differences between difference opens, that would show some sort of sentiment of global player. it would be easier targeting US Open by just converting to US open time. 



### raw data, add ny time, select us open

1. convert date as index 
2. convert utc to new york time for easier filtering


I've checked the raw data from pepperstone export. Then I also checked the graph from November 4 2025 in Pepperstone. I noticed, that the typical us open structure was at 15.30 my time, when i set it to UTC+1. (It used to be UTC+2 before DTS)
Then I've checked the Volume bump and it's structure -> Volume 84 to Volume 176 and I saw that exact value bump 1 hour earlier in the exported data. That means, the data that I've written with the automation code writes regular UTC time. 

This is good, because then I just need to assign UTC to the raw datetime. 


```
code is supposed to be here 
```



In [17]:
import pytz
from edge_tools.time import preprocess_time 

# could use split_us_market_hours from edge_tools.us_open import split_us_market_hours
# but here we will do it manually for clarity
data = preprocess_time(data)



# Define US open/close bounds
open_time = pd.to_datetime("09:30").time()
close_time = pd.to_datetime("16:00").time()

# Filter only market hours
data['is_us_market_open'] = data['ny_time_only'].between(open_time, close_time)
data_us_open = data[data['is_us_market_open']]

In [18]:
data

,open,high,low,close,volume,ny_time,ny_hour,ny_minute,ny_time_only,tokyo_time,...,tokyo_time_only,london_time,london_hour,london_minute,london_time_only,shanghai_time,shanghai_hour,shanghai_minute,shanghai_time_only,is_us_market_open
time,,,,,,,,,,,,,,,,,,,,,
2025-10-28 08:46:00+00:00,6873.8,6873.8,6873.3,6873.8,24,2025-10-28 04:46:00-04:00,4,46,04:46:00,2025-10-28 17:46:00+09:00,...,17:46:00,2025-10-28 08:46:00+00:00,8,46,08:46:00,2025-10-28 16:46:00+08:00,16,46,16:46:00,False
2025-10-28 08:47:00+00:00,6874.0,6874.8,6874.0,6874.8,26,2025-10-28 04:47:00-04:00,4,47,04:47:00,2025-10-28 17:47:00+09:00,...,17:47:00,2025-10-28 08:47:00+00:00,8,47,08:47:00,2025-10-28 16:47:00+08:00,16,47,16:47:00,False
2025-10-28 08:48:00+00:00,6874.5,6874.8,6874.1,6874.5,12,2025-10-28 04:48:00-04:00,4,48,04:48:00,2025-10-28 17:48:00+09:00,...,17:48:00,2025-10-28 08:48:00+00:00,8,48,08:48:00,2025-10-28 16:48:00+08:00,16,48,16:48:00,False
2025-10-28 08:49:00+00:00,6874.3,6875.5,6874.3,6875.5,16,2025-10-28 04:49:00-04:00,4,49,04:49:00,2025-10-28 17:49:00+09:00,...,17:49:00,2025-10-28 08:49:00+00:00,8,49,08:49:00,2025-10-28 16:49:00+08:00,16,49,16:49:00,False
2025-10-28 08:50:00+00:00,6875.3,6876.8,6875.3,6876.5,26,2025-10-28 04:50:00-04:00,4,50,04:50:00,2025-10-28 17:50:00+09:00,...,17:50:00,2025-10-28 08:50:00+00:00,8,50,08:50:00,2025-10-28 16:50:00+08:00,16,50,16:50:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-04 08:44:00+00:00,6790.1,6792.2,6790.1,6791.4,96,2025-11-04 03:44:00-05:00,3,44,03:44:00,2025-11-04 17:44:00+09:00,...,17:44:00,2025-11-04 08:44:00+00:00,8,44,08:44:00,2025-11-04 16:44:00+08:00,16,44,16:44:00,False
2025-11-04 08:45:00+00:00,6791.8,6792.4,6790.4,6790.7,90,2025-11-04 03:45:00-05:00,3,45,03:45:00,2025-11-04 17:45:00+09:00,...,17:45:00,2025-11-04 08:45:00+00:00,8,45,08:45:00,2025-11-04 16:45:00+08:00,16,45,16:45:00,False
2025-11-04 08:46:00+00:00,6790.8,6791.8,6789.7,6790.2,116,2025-11-04 03:46:00-05:00,3,46,03:46:00,2025-11-04 17:46:00+09:00,...,17:46:00,2025-11-04 08:46:00+00:00,8,46,08:46:00,2025-11-04 16:46:00+08:00,16,46,16:46:00,False


In [10]:
data_us_open[(data_us_open['ny_hour'] == 9) & (data_us_open['ny_minute'] == 30)][["ny_time"]]

,ny_time
time,
2025-10-28 13:30:00+00:00,2025-10-28 09:30:00-04:00
2025-10-29 13:30:00+00:00,2025-10-29 09:30:00-04:00
2025-10-30 13:30:00+00:00,2025-10-30 09:30:00-04:00
2025-10-31 13:30:00+00:00,2025-10-31 09:30:00-04:00
2025-11-03 14:30:00+00:00,2025-11-03 09:30:00-05:00


## Fun Fact

A week ago, when I went out with monty and Edward, there was this DTS happening, giving us one more hour. 
I've heard that New York, DTS happens a week later, so there is always this confusion. Now after checking UTC trading data, i can see how NY Time changes and what UTC time it also is. 

## switch for us open data only or full data for subsequent analysis

In [32]:
data = data_us_open 

### resample to day data

In [33]:
# Resample to daily OHLCV
daily = data.resample('1D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})
daily.dropna(inplace=True)  # Drop any rows with NaN values
daily

,open,high,low,close,volume
time,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902
...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748


## ATR (Average True Range) Analysis 
### ATR

In [34]:
# Compute ATR with a 14-period window (default Wilder’s smoothing)
daily["ATR_14"] = ta.atr(
    high=daily["high"], 
    low=daily["low"], 
    close=daily["close"], 
    length=14
)

# Normalize to price: ATR% = ATR / Close * 100
daily["ATR_pct"] = daily["ATR_14"] / daily["close"] * 100

daily.tail()[["ATR_14", "ATR_pct"]]

,ATR_14,ATR_pct
time,,
2025-08-14 00:00:00+00:00,55.670707,0.860670
2025-08-15 00:00:00+00:00,54.272799,0.841517
2025-08-18 00:00:00+00:00,51.631885,0.800296
2025-08-19 00:00:00+00:00,51.986750,0.809977
2025-08-20 00:00:00+00:00,53.373411,0.834233


### finding unique days in the data

In [35]:
# Create a Series of unique dates in MM-DD format
unique_dates = pd.Series(daily.index.strftime("%m-%d").unique())
unique_dates.sort_values(inplace=True)
unique_dates

# so unique days are 366 throughout 5 years now .

4      01-02
5      01-03
261    01-04
262    01-05
6      01-06
       ...  
1      12-27
259    12-28
260    12-29
2      12-30
3      12-31
Name: time, Length: 364, dtype: object

### add date details to daily table 

In [36]:
# Ensure the index is a DatetimeIndex
daily = daily.sort_index()
if not isinstance(daily.index, pd.DatetimeIndex):
    daily.index = pd.to_datetime(daily.index)

# Calendar features
iso = daily.index.isocalendar()  # returns a DataFrame: year, week, day
daily["year"]        = iso["year"].astype(int)
daily["week"]        = iso["week"].astype(int)       # ISO week number (1–53)
daily["iso_weekday"] = iso["day"].astype(int)        # ISO: Mon=1 ... Sun=7

# Also handy:
daily["month"]       = daily.index.month             # 1–12
daily["day"]         = daily.index.day               # 1–31
daily["weekday"]     = daily.index.weekday           # Mon=0 ... Sun=6 (Python style)
daily["month_name"]  = daily.index.month_name()
daily["weekday_name"]= daily.index.day_name()


### add return

In [37]:
# Example: compute daily returns first
daily["ret"] = daily["close"].pct_change() 
daily["ret_pct"] = daily["ret"] * 100  # Convert to percentage
daily

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct
time,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112


### filter only weekdays

In [38]:
# Filter to weekdays only (Mon–Fri)
daily = daily[daily.index.weekday < 5]  # Mon–Fri only

daily

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct
time,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112


### add rolling quantile and add atr segments

In [39]:
import numpy as np

# Create ATR buckets based on quantiles
q33 = daily["ATR_pct"].rolling(252).quantile(0.33)
q66 = daily["ATR_pct"].rolling(252).quantile(0.66)

# Assign buckets based on quantiles
# "low" for below 33rd percentile, "mid" for between 33rd and 66th, "high" for above 66th
daily["ATR_bucket"] = np.where(daily["ATR_pct"] <= q33, "low",
                        np.where(daily["ATR_pct"] <= q66, "mid", "high"))


daily

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct,ATR_bucket
time,,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN,high
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087,high
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226,high
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408,high
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536,low
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194,low
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112,low


In [40]:
# Check the distribution of ATR buckets
daily["ATR_bucket"].value_counts(normalize=True)


ATR_bucket
high    0.444444
low     0.370370
mid     0.185185
Name: proportion, dtype: float64

### some statistics 

In [41]:
daily.groupby(["weekday","ATR_bucket"])["ret_pct"].mean().unstack()


ATR_bucket,high,low,mid
weekday,,,
0,0.020498,0.161673,0.213085
1,0.231300,-0.076738,-0.033707
2,0.103607,0.048630,0.042738
3,-0.133886,0.150466,-0.013748
4,-0.042535,0.125166,0.069687


In [42]:
daily.groupby(["year","ATR_bucket"])["ret_pct"].mean().unstack()

ATR_bucket,high,low,mid
year,,,
2019,-0.003087,NaN,NaN
2020,0.078292,NaN,NaN
2021,0.151378,0.075640,0.130202
2022,-0.083856,0.065122,-0.093481
2023,NaN,0.079369,0.127244
2024,0.116440,0.073936,0.082258
2025,0.022091,0.096787,0.093833


### relative volume

In [45]:
n = 20  # lookback
daily['RVOL'] = daily['volume'] / daily['volume'].rolling(n).mean()

daily[['volume','RVOL']]

,volume,RVOL
time,,
2019-12-26 00:00:00+00:00,4754,NaN
2019-12-27 00:00:00+00:00,10270,NaN
2019-12-30 00:00:00+00:00,16344,NaN
2019-12-31 00:00:00+00:00,13832,NaN
2020-01-02 00:00:00+00:00,13902,NaN
...,...,...
2025-08-14 00:00:00+00:00,26268,1.053852
2025-08-15 00:00:00+00:00,23922,0.949719
2025-08-18 00:00:00+00:00,18748,0.735709


$DMR = \frac{|Close - Open|}{High - Low}$

👉 Interpretation:
	•	Measures how much net distance the market traveled compared to how much was available.
	•	High DMR (≥ 0.6) → the day actually went somewhere (trend).
	•	Low DMR (≤ 0.3) → the day expanded in both directions but closed near the open (chop/fakeouts).

📈 Why useful:
	•	Filters trend days vs range days.
	•	Tells you: “Did we make directional progress, or just wiggle in place?”

In [49]:
daily['DMR'] = (daily['close'] - daily['open']).abs() / (daily['high'] - daily['low'])
daily 

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct,ATR_bucket,RVOL,Gap_pct,CLV,DMR
time,,,,,,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN,high,NaN,NaN,0.914286,0.892857
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087,high,NaN,0.265498,0.368794,0.617021
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226,high,NaN,0.024698,0.266932,0.681275
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408,high,NaN,-0.238923,0.939394,0.752525
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241,high,NaN,0.448916,0.928571,0.495798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536,low,1.053852,-0.346122,0.712121,0.575758
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194,low,0.949719,0.151508,0.180055,0.795014
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112,low,0.735709,-0.021707,0.612717,0.208092


🧩 2. CLV – Close Location Value

Formula:

$CLV = \frac{Close - Low}{High - Low}$

👉 Interpretation:
	•	Measures where the close landed inside the day’s range.
	•	CLV ≈ 1 → closed near highs → bullish dominance.
	•	CLV ≈ 0 → closed near lows → bearish dominance.
	•	CLV ≈ 0.5 → closed mid-range → balance day.

📈 Why useful:
	•	Adds directional bias (up vs down).
	•	Tells you: “Did buyers or sellers finish with control?”

In [50]:
daily['CLV'] = (daily['close'] - daily['low']) / (daily['high'] - daily['low'])
daily

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct,ATR_bucket,RVOL,Gap_pct,CLV,DMR
time,,,,,,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN,high,NaN,NaN,0.914286,0.892857
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087,high,NaN,0.265498,0.368794,0.617021
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226,high,NaN,0.024698,0.266932,0.681275
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408,high,NaN,-0.238923,0.939394,0.752525
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241,high,NaN,0.448916,0.928571,0.495798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536,low,1.053852,-0.346122,0.712121,0.575758
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194,low,0.949719,0.151508,0.180055,0.795014
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112,low,0.735709,-0.021707,0.612717,0.208092


In [47]:
daily['Gap_pct'] = (daily['open'] - daily['close'].shift(1)) / daily['close'].shift(1) * 100

daily


,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct,ATR_bucket,RVOL,Gap_pct
time,,,,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN,high,NaN,NaN
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087,high,NaN,0.265498
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226,high,NaN,0.024698
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408,high,NaN,-0.238923
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241,high,NaN,0.448916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536,low,1.053852,-0.346122
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194,low,0.949719,0.151508
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112,low,0.735709,-0.021707


In [57]:
# Classify regimes using the edge_tools.regimes module

from edge_tools.regimes import classify_regimes

labeled = classify_regimes(daily)

# Inspect frequency of regimes
labeled['Regime'].value_counts()

# # Example: filter only HighVol_TrendUp days
# high_trend_up = labeled[labeled['Regime'] == 'HighVol_TrendUp']

labeled

,open,high,low,close,volume,ATR_14,ATR_pct,year,week,iso_weekday,month,day,weekday,month_name,weekday_name,ret,ret_pct,ATR_bucket,RVOL,Gap_pct,CLV,DMR,ATR,VolBucket,TrendDir,Trendiness,Regime
time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-26 00:00:00+00:00,3226.7,3240.4,3226.4,3239.2,4754,NaN,NaN,2019,52,4,12,26,3,December,Thursday,NaN,NaN,high,NaN,NaN,0.914286,0.892857,NaN,NaN,TrendUp,Trend,nan_TrendUp
2019-12-27 00:00:00+00:00,3247.8,3248.0,3233.9,3239.1,10270,NaN,NaN,2019,52,5,12,27,4,December,Friday,-0.000031,-0.003087,high,NaN,0.265498,0.368794,0.617021,NaN,NaN,Balanced,Trend,nan_Trend
2019-12-30 00:00:00+00:00,3239.9,3241.2,3216.1,3222.8,16344,NaN,NaN,2020,1,1,12,30,0,December,Monday,-0.005032,-0.503226,high,NaN,0.024698,0.266932,0.681275,NaN,NaN,TrendDown,Trend,nan_TrendDown
2019-12-31 00:00:00+00:00,3215.1,3231.2,3211.4,3230.0,13832,NaN,NaN,2020,1,2,12,31,1,December,Tuesday,0.002234,0.223408,high,NaN,-0.238923,0.939394,0.752525,NaN,NaN,TrendUp,Trend,nan_TrendUp
2020-01-02 00:00:00+00:00,3244.5,3258.0,3234.2,3256.3,13902,NaN,NaN,2020,1,4,1,2,3,January,Thursday,0.008142,0.814241,high,NaN,0.448916,0.928571,0.495798,NaN,NaN,TrendUp,Mixed,nan_Mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14 00:00:00+00:00,6449.3,6477.8,6444.8,6468.3,26268,55.670707,0.860670,2025,33,4,8,14,3,August,Thursday,-0.000525,-0.052536,low,1.053852,-0.346122,0.712121,0.575758,55.670707,LowVol,TrendUp,Mixed,LowVol_Mixed
2025-08-15 00:00:00+00:00,6478.1,6479.0,6442.9,6449.4,23922,54.272799,0.841517,2025,33,5,8,15,4,August,Friday,-0.002922,-0.292194,low,0.949719,0.151508,0.180055,0.795014,54.272799,LowVol,TrendDown,Trend,LowVol_TrendDown
2025-08-18 00:00:00+00:00,6448.0,6458.3,6441.0,6451.6,18748,51.631885,0.800296,2025,34,1,8,18,0,August,Monday,0.000341,0.034112,low,0.735709,-0.021707,0.612717,0.208092,51.631885,LowVol,Balanced,Range,LowVol_Range


In [59]:
# Assuming 'labeled' has columns 'Regime', 'month', and 'year'
counts = labeled.groupby(['year', 'month'])['TrendDir'].value_counts().unstack(fill_value=0)
counts

TrendDir    Balanced  TrendDown  TrendUp
year month                              
2019 12            1          0        1
2020 1             6          6       10
     2             2          8       10
     3             3         10        9
     4             4          7       10
...              ...        ...      ...
2025 5             6          7        9
     6             5          8        8
     7             6          7       10
     8             4          4        6
     12            1          1        0

[70 rows x 3 columns]

## helping analysis
### plot using ny time



In [23]:
import plotly.graph_objects as go
from datetime import datetime

# Select the prev day
unique_dates = sorted(data.index.date)

# Use the last date in the sorted list as the previous day
# Adjust the index to select the desired previous day
prev_day = pd.Series(data.index.date).unique()[-2]
prev_day = datetime(2024, 12, 30).date()  # Example date, adjust as needed

print(prev_day)
prev_df = data[data["ny_time"].dt.date == prev_day]


fig = go.Figure(data=[
    go.Candlestick(
        x=prev_df["ny_time"],
        open=prev_df['open'],
        high=prev_df['high'],
        low=prev_df['low'],
        close=prev_df['close'],
        name='Candles'
    ),
    go.Bar(
        x=prev_df["ny_time"],
        y=prev_df['volume'],
        name='Volume',
        marker_color='#1a237e',
        yaxis='y2'
    )
])

fig.update_layout(
    title=f"S&P 500 Minute Candles for {prev_day}",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="Price"),
    yaxis2=dict(
        title="Volume",
        overlaying="y",
        side="right",
        showgrid=False,
        range=[0, prev_df['volume'].max() * 4]  # Adjust multiplier for better scaling
    ),
    legend=dict(orientation="h")
)

fig.show()

2024-12-30
